In [3]:
import re
from bs4 import BeautifulSoup as bs

# метод получения названия статьи
def get_title(data):
    text = data.find('h1')
    title = text.text.strip() if text else ''
    return title

# метод получения текста статьи
def get_text(data):
    r = re.compile(r"^\s+", re.MULTILINE)
    [s.extract() for s in data.findAll('code')]
    text = r.sub("", data.find('div', 'post__text').text)
    return text

In [4]:
from zipfile import ZIP_DEFLATED, ZipFile

# считываем файлы из прошлого задания и добавляем текст в список
data = []
counter = 0
with ZipFile('../task1/pages.zip', 'r', ZIP_DEFLATED) as zipFile:
    for info in zipFile.infolist():
        file_data = zipFile.read(info.filename)
        html_text = bs(file_data, features="html.parser")
        title = get_title(html_text)
        counter += 1
        text = get_title(html_text) + "\n" + get_text(html_text)
        data.append(text)
print(f'added texts to data {len(data)}')

added texts to data 100


In [ ]:
# сохраняем только тексты в новом архиве (на будущее)
counter = 0
with ZipFile('texts.zip', 'w', ZIP_DEFLATED) as zipFile:
    for text in data:
        counter += 1
        zipFile.writestr(f'document_{counter}.txt', text)

In [3]:
import nltk
import string

# добавляем стоп-слова и знаки пунктуации для последующей токенизации
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words("russian")
for ch in string.punctuation:
    stop_words.append(ch)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rusla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
from nltk import word_tokenize

# метод получения токенов из текста.
# обработка: 
# разбиваем текст на токены, 
# переводим токены в нижний регистр, 
# убираем стоп-слова, 
# оставляем только aplha-numeric значения

def get_tokens(text):
    tokens = word_tokenize(text.replace("-", " "), language="russian")
    tokens = [i.lower() for i in tokens]
    tokens = ([i for i in tokens if i not in stop_words])
    tokens = [i for i in tokens if i != "" and i.isalpha()]
    return tokens

In [2]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# метод для лемматизации токенов (создаем словарь, к каждой нормальной форме сопоставляем список из исходных токенов)
def lemmatize_tokens(tokens):
    lemmas = {}
    for token in tokens:
        lemma = morph.parse(token)[0].normal_form
        if lemmas.get(lemma, None) is None:
            lemmas[lemma] = [token]
        else:
            lemmas[lemma].append(token)
    return lemmas

In [5]:
import io
from zipfile import ZIP_DEFLATED, ZipFile

# считываем тексты
data = []
with ZipFile('texts.zip', 'r', ZIP_DEFLATED) as zipFile:
    for info in zipFile.infolist():
        with io.TextIOWrapper(zipFile.open(info.filename), encoding="utf-8") as f:
            data.append(f.read())
print(f'posts length = {len(data)}')

# создаем список токенов, с помощью метода выше добавляем токены из каждого текста поста
tokens = []
for post in data:
    tokens += get_tokens(post)
    
# убираем дупликаты
tokens = list(dict.fromkeys(tokens))

# записываем токены в файл
with open('tokens.txt', "w", encoding="utf-8") as file:
    for token in tokens:
        file.write(token + "\n")
print(f'tokens length = {len(tokens)}')

# лемматизируем токены
lemmas = lemmatize_tokens(tokens)

# записываем нормальную форму и исходные токены в файл
with open('lemmas.txt', "w", encoding="utf-8") as file:
    for lemma, tokens in lemmas.items():
        file.write(lemma + " " + " ".join(tokens) + "\n")
print(f'lemmas length = {len(lemmas)}')

posts length = 100
tokens length = 30920
lemmas length = 15029
